
Driven Data Water Contest

Import needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#import data
train = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/datadrivenH2O/train.csv')
target = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/datadrivenH2O/target.csv')
test = pd.read_csv('/Users/ericp/OneDrive/Documents/GitHub/datadrivenH2O/test.csv')
train_id = train['id']
test_id = test['id']
train = train.drop(['id'], axis = 1)
test = test.drop(['id'], axis = 1)
colnames = train.columns
train_shape = train.shape[0]
train.head(10)


amount_tsh date_recorded               funder  gps_height     installer  \
0      6000.0    14/03/2011                Roman        1390         Roman   
1         0.0     6/03/2013              Grumeti        1399       GRUMETI   
2        25.0    25/02/2013         Lottery Club         686  World vision   
3         0.0    28/01/2013               Unicef         263        UNICEF   
4         0.0    13/07/2011          Action In A           0       Artisan   
5        20.0    13/03/2011  Mkinga Distric Coun           0           DWE   
6         0.0     1/10/2012                 Dwsp           0          DWSP   
7         0.0     9/10/2012                Rwssp           0           DWE   
8         0.0     3/11/2012             Wateraid           0     Water Aid   
9         0.0     3/08/2011          Isingiro Ho           0       Artisan   

   longitude   latitude              wpt_name  num_private  \
0  34.938093  -9.856322                  none            0   
1  34.698766  -2.147466              Zahanati            0   
2  37.460664  -3.821329           Kwa Mahundi            0   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0   
4  31.130847  -1.825359               Shuleni            0   
5  39.172796  -4.765587                Tajiri            0   
6  33.362410  -3.766365            Kwa Ngomho            0   
7  32.620617  -4.226198          Tushirikiane            0   
8  32.711100  -5.146712     Kwa Ramadhan Musa            0   
9  30.626991  -1.257051               Kwapeto            0   

                     basin  ... payment_type water_quality  quality_group  \
0               Lake Nyasa  ...     annually          soft           good   
1            Lake Victoria  ...    never pay          soft           good   
2                  Pangani  ...   per bucket          soft           good   
3  Ruvuma / Southern Coast  ...    never pay          soft           good   
4            Lake Victoria  ...    never pay          soft           good   
5                  Pangani  ...   per bucket         salty          salty   
6                 Internal  ...    never pay          soft           good   
7          Lake Tanganyika  ...      unknown         milky          milky   
8          Lake Tanganyika  ...    never pay         salty          salty   
9            Lake Victoria  ...    never pay          soft           good   

       quantity quantity_group                source           source_type  \
0        enough         enough                spring                spring   
1  insufficient   insufficient  rainwater harvesting  rainwater harvesting   
2        enough         enough                   dam                   dam   
3           dry            dry           machine dbh              borehole   
4      seasonal       seasonal  rainwater harvesting  rainwater harvesting   
5        enough         enough                 other                 other   
6        enough         enough           machine dbh              borehole   
7        enough         enough          shallow well          shallow well   
8      seasonal       seasonal           machine dbh              borehole   
9        enough         enough          shallow well          shallow well   

  source_class              waterpoint_type waterpoint_type_group  
0  groundwater           communal standpipe    communal standpipe  
1      surface           communal standpipe    communal standpipe  
2      surface  communal standpipe multiple    communal standpipe  
3  groundwater  communal standpipe multiple    communal standpipe  
4      surface           communal standpipe    communal standpipe  
5      unknown  communal standpipe multiple    communal standpipe  
6  groundwater                    hand pump             hand pump  
7  groundwater                    hand pump             hand pump  
8  groundwater                    hand pump             hand pump  
9  groundwater                    hand pump             hand pump  

[10 r

In [6]:
#look at train / test shapes
print(train.shape)
print(test.shape)
print(target.shape)

(59400, 39)
(14850, 39)
(59400, 2)


In [7]:
#look at the value_counts of the target variable
target['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [9]:
#LabelEncode target variable
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target['status_group'] = le.fit_transform(target['status_group'])
target['status_group'].value_counts()
#functional = 0
#non-functional = 1
#functional needs repair = 2

0    32259
2    22824
1     4317
Name: status_group, dtype: int64

In [ ]:
#look at correlations of variables with target
#check for correlation between target and predictors
target_corr = list()

for c, v in enumerate(train, start = 1):
    target_corr.append(target.corrwith(train[v], method = 'spearman'))
    
target_corr = pd.Series(data = target_corr, index = train.columns, name = 'correlation')
target_corr = abs(target_corr)
target_corr
#not a lot highly correlated with the target

In [ ]:
#combine data
combine = pd.concat([train, test],  axis = 0).reset_index(drop = True)

In [ ]:
#look for missing values
miss_vals = pd.Series(combine.isnull().sum(), name = 'PctMissing')
miss_vals = miss_vals[miss_vals!=0]
miss_vals = miss_vals.sort_values(ascending = False)
print(miss_vals)

In [ ]:
#pct missing
miss_vals / len (combine)

In [ ]:
#these variables all seem to be about area / region. Might be best to use the mode of the region they're in
combine['subvillage'] = combine.groupby('region')['subvillage'].transform(lambda x:x.fillna(x.mode()[0]))
combine['public_meeting'] = combine.groupby('region')['public_meeting'].transform(lambda x:x.fillna(x.mode()[0]))
combine['permit'] = combine.groupby('region')['permit'].transform(lambda x:x.fillna(x.mode()[0]))
combine['funder'] = combine.groupby('region')['funder'].transform(lambda x:x.fillna(x.mode()[0]))
combine['installer'] = combine.groupby('region')['funder'].transform(lambda x:x.fillna(x.mode()[0]))
combine['scheme_management'] = combine.groupby('region')['scheme_management'].transform(lambda x:x.fillna(x.mode()[0]))

In [ ]:
#scheme_management and scheme_name seem to be redundant. Lots of missing values for scheme_name. Will delete.
#region and region_code same. Will delete region
#extraction_type_group seems identical to extraction_type
combine = combine.drop(['scheme_name', 'region', 'extraction_type_group'], axis = 1)

In [ ]:
#check missing values
miss_vals = pd.Series(combine.isnull().sum(), name = 'PctMissing')
miss_vals = miss_vals[miss_vals!=0]
miss_vals = miss_vals.sort_values(ascending = False)
print(miss_vals)

In [ ]:
dtyp = pd.Series(combine.dtypes, name = 'dtype')
dtyp

In [ ]:
#some items as int are actually objects
combine['construction_year'] = combine['construction_year'].astype('object')

In [ ]:
#check correlations between variables. See if there's some that can be deleted
f = plt.figure(figsize=(8, 8))
plt.matshow(combine.corr(), fignum=f.number)
plt.yticks(range(combine.select_dtypes(['number']).shape[1]), combine.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
corr_df = combine.corr()
corr_df
#all correlations quite low. Keep all predictors.

In [ ]:
#label encode columns where there is structure (i.e. levels)
le_columns = ['date_recorded', 'permit']

In [ ]:
train = combine[:train_shape]
test = combine[train_shape:]
print(train.shape)
print(test.shape)

In [ ]:
#import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
#create metric to determine accuracy
from sklearn.model_selection import KFold, cross_val_score
def accuracy(model, X, y, n = 5):
    kf = KFold(n, random_state = 1, shuffle = True)
    acc = cross_val_score(model, X, y, scoring = 'accuracy', cv = kf)
    return acc    

Create Base Models

In [ ]:
#create base models
lgb = LGBMClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()

Base Model Scores

In [ ]:
score = accuracy(rf, train, target)